In [2]:
%matplotlib notebook
import mne
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from scipy.signal import butter, cheby1, filtfilt

from scipy.signal import spectrogram, stft, istft, check_NOLA
import pickle

# IMPORT DATA AND PREPROCESS

In [19]:
# 31, 35, 38 and test folders for good data
# odd scalp on left, even on right

# SPECIFY PATIENT AND SCALP TARGET ELECTRODE
patient = 'UFSEEG031'
targetScalpElectrodes = [] # assigned below

# SPECIFY ARTIFACT ELECTRODES FROM WORD FILES
artifactElectrodes = {}
artifactElectrodes['UFSEEG031'] = ['LTP7', 'LTP8', 'LAH11', 'LAH12', 'LPH10', 'LPH11', 'LPH12','LOF15', 'LOF16']

filepath = '/blue/gkalamangalam/ALLDATA/SEEG/%s/SEEG/EDF/EttingerFiles/Sz5File.edf' % patient

raw = mne.io.read_raw_edf(filepath,preload=True)
sfreq = int(raw.info['sfreq'])

scalpElectrodes = {}
scalpElectrodes[patient] = [i for i in raw.ch_names if len(i) == 2 and i !='PR']
targetScalpElectrodes = scalpElectrodes[patient]
print()
print(scalpElectrodes)
print(raw)
print(raw.info)

Extracting EDF parameters from /blue/gkalamangalam/ALLDATA/SEEG/UFSEEG031/SEEG/EDF/EttingerFiles/Sz5File.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 828031  =      0.000 ...   808.624 secs...

{'UFSEEG031': ['F7', 'F8', 'F3', 'F4', 'C3', 'C4', 'P7', 'P8', 'P3', 'P4']}
<RawEDF | Sz5File.edf, 148 x 828032 (808.6 s), ~935.1 MB, data loaded>
<Info | 7 non-empty values
 bads: []
 ch_names: LTP1, LTP2, LTP3, LTP4, LTP5, LTP6, LTP7, LTP8, LAM1, LAM2, ...
 chs: 148 EEG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 512.0 Hz
 meas_date: 2001-01-01 00:53:11 UTC
 nchan: 148
 projs: []
 sfreq: 1024.0 Hz
>


In [20]:
# DISCARD ALL CHANNELS EXCEPT GOOD SEEG CHANNELS AND THE SCALP TARGETS

channels = [i for i in raw.ch_names if i not in artifactElectrodes[patient] and i[0] in {'L', 'R'}] + targetScalpElectrodes
raw.pick_channels(channels)#.plot(duration=5.0, n_channels=20);

nScalp = len(targetScalpElectrodes)
nDepth = len(channels) - nScalp

In [21]:
# LOWPASS FILTER THE DATA, SUBSAMPLE THE DATA, SCALE ALL CHANNELS, AND EXTRACT TO NUMPY ARRAY

filterWindow = 64
subsampleFreq = filterWindow * 2   # FINAL FREQUENCY IN HERTZ AFTER SUBSAMPLING
filterOrder = 5
goodTimes = [] # set this below

df = raw.to_data_frame().drop(labels=['time'], axis=1)
data = df.to_numpy()

# SHOULD WE SCALE HERE OR AFTER FILTER AND SUBSAMPLE?
scaler = StandardScaler()
data = scaler.fit_transform(data)
goodTimes = range(int(data.shape[0] * .75))
data = data[goodTimes,:]

b, a = butter(filterOrder, filterWindow, btype='lowpass', fs = sfreq)
data = filtfilt(b, a, data, axis=0)
    
dataSubsampled = data[::sfreq // subsampleFreq,:]

# SEE QUESTION ABOVE...
#scaler = StandardScaler()
#dataSubsampled = scaler.fit_transform(dataSubsampled)

pd.DataFrame(dataSubsampled, columns=df.columns)

LTP1      LTP2      LTP3      LTP4      LTP5      LTP6      LAM1  \
0      0.923163  0.877558  2.007037  2.142583  2.431144  0.068585  0.406731   
1      0.967851  0.895719  2.078280  2.194973  2.497465  0.160408  0.480558   
2      0.950634  0.889677  2.030407  2.150442  2.424340  0.068409  0.491443   
3      0.924689  0.873459  1.992786  2.115316  2.383078 -0.002456  0.459827   
4      0.975040  0.910185  2.092402  2.152423  2.451111  0.010998  0.449090   
...         ...       ...       ...       ...       ...       ...       ...   
77623  0.624531  0.745006  1.441203  1.068362  1.378554 -0.158661  0.447754   
77624  0.543735  0.683965  1.149104  0.865553  1.197851 -0.091057  0.354996   
77625  0.634033  0.815001  1.174487  0.893286  1.257674  0.243780  0.423135   
77626  0.624417  0.750482  0.866908  0.685005  1.008557  0.339767  0.320995   
77627  0.719435  0.784803  0.752821  0.610069  0.916843  0.461246  0.348404   

           LAM2      LAM3      LAM4  ...        F7        F8        F3  \
0      0.768738  0.257738  1.026236  ... -1.076997  0.808610 -0.836579   
1      0.808264  0.278556  1.108244  ... -0.902773  0.913212 -0.615324   
2      0.779953  0.272490  1.104432  ... -0.991131  0.917049 -0.693842   
3      0.817384  0.252441  1.057774  ... -1.068590  0.918067 -0.747148   
4      0.799488  0.272411  1.087969  ... -1.061989  1.012355 -0.633206   
...         ...       ...       ...  ...       ...       ...       ...   
77623  0.120075  0.090933  0.868139  ... -0.732945  0.398884 -0.277111   
77624  0.048507  0.010678  0.669410  ... -1.052829 -0.162764 -0.964954   
77625  0.098308  0.069546  0.776280  ... -0.620866  0.169789 -0.080186   
77626  0.023991 -0.038705  0.577973  ... -0.807264 -0.133751 -0.704837   
77627  0.077212  0.026648  0.606527  ... -0.384613  0.313692 -0.100831   

             F4        C3        C4        P7        P8        P3        P4  
0      0.247534 -0.452151  0.555819  0.060836  0.857042  0.304849  0.585604  
1      0.468168 -0.177805  0.791495  0.203334  0.916803  0.500818  0.692449  
2      0.507542 -0.279770  0.679294  0.043708  0.652739  0.352665  0.585400  
3      0.513551 -0.432375  0.567513 -0.193670  0.467857  0.138745  0.462746  
4      0.797708 -0.245303  0.753528 -0.113916  0.609422  0.208804  0.568187  
...         ...       ...       ...       ...       ...       ...       ...  
77623  0.278927  0.290293  0.518874  0.475420  0.235904  0.572566  0.328553  
77624 -0.419954 -0.653330 -0.212376 -0.435585 -0.289683 -0.270588  0.005233  
77625  0.293646  0.152075  0.289111  0.247576  0.422988  0.367409  0.375888  
77626 -0.267866 -0.704884 -0.409542 -0.144113 -0.170908 -0.217600  0.005653  
77627  0.422563  0.235346  0.173756  0.368175  0.543122  0.260659  0.291934  

[77628 rows x 97 columns]

In [22]:
# PARTITION TIME SERIES INTO CONTIGUOUS TRAIN AND VALIDATION BLOCKS
# OTHERWISE WHEN VALID SET IS RANDOMLY DISTRIBUTED, IMPLICIT OVERFITTING OCCURS DUE TO TEMPORAL DEPENDENCY
# EACH BLOCK (SPECIFIED IN SECONDS) IS DIVIDED INTO TRAIN (SPECIFIED BY FRACTION, COMES FIRST) AND VALIDIDATION (COMES LAST)

def timeseriesTrainValidSplit(secondsInBlock, totalSeconds, trainFraction, subsampleFreq):
    nBlock = int(totalSeconds / secondsInBlock)
    samplesPerBlock = subsampleFreq * secondsInBlock
    trainIndexProto = np.arange(0, samplesPerBlock * trainFraction, dtype=int)
    validIndexProto = np.arange(samplesPerBlock * trainFraction, samplesPerBlock, dtype=int)

    trainIndexBlocks = [(trainIndexProto + (i * samplesPerBlock)).astype(int) for i in range(nBlock)]
    validIndexBlocks = [(validIndexProto + (i * samplesPerBlock)).astype(int) for i in range(nBlock)]

    trainIndices = np.concatenate(trainIndexBlocks).astype(int)
    validationIndices = np.concatenate(validIndexBlocks).astype(int)
    #return trainIndices, validationIndices
    return trainIndexBlocks, validIndexBlocks, trainIndices, validationIndices

def timeDomainDataMake(indexBlocks, halfWindow, dataSubsampled):
    xList = []
    yList = []
    _, nChannels = dataSubsampled.shape
    for thisBlock in indexBlocks:
        thisData = np.vstack([np.zeros((halfWindow, nChannels)), 
                             dataSubsampled[thisBlock,:], 
                             np.zeros((halfWindow, nChannels))])
        
        for t in range(0, len(thisBlock)):
            thisX = thisData[t:t + (2 * halfWindow) + 1,0:nDepth].flatten()
            thisY = thisData[t + halfWindow, nDepth:]
            xList.append(thisX)
            yList.append(thisY)

    xTimeDomain = np.stack(xList, axis = 0)
    #yTimeDomain = np.expand_dims(np.array(yList), axis=1)
    yTimeDomain = np.stack(yList, axis = 0)
    return xTimeDomain, yTimeDomain

# INDICES FOR TRAIN/VALIDIDATION SPLIT

In [23]:
secondsInBlock = 5
trainFraction = 0.0
totalSeconds = int(dataSubsampled.shape[0]/subsampleFreq)

if trainFraction > 0.0:
    trainIndexBlocks, validIndexBlocks, trainIndices, validationIndices = timeseriesTrainValidSplit(secondsInBlock, 
                                                                                                    totalSeconds, 
                                                                                                    trainFraction, 
                                                                                                    subsampleFreq)

# TIME DOMAIN DATA

In [24]:
halfWindowSeconds = .25

halfWindowSamples = int(halfWindowSeconds * subsampleFreq)

if trainFraction > 0.0:
    xTrainTimeDomain, yTrainTimeDomain = timeDomainDataMake(trainIndexBlocks, halfWindowSamples, dataSubsampled)
    xValidTimeDomain, yValidTimeDomain = timeDomainDataMake(validIndexBlocks, halfWindowSamples, dataSubsampled)

else:
    validIndexBlocks = [np.array(range(0, dataSubsampled.shape[0]))]
    xValidTimeDomain, yValidTimeDomain = timeDomainDataMake(validIndexBlocks, halfWindowSamples, dataSubsampled)
    xTrainTimeDomain, yTrainTimeDomain = np.array([]), np.array([])
    
print(xTrainTimeDomain.shape, yTrainTimeDomain.shape, xValidTimeDomain.shape, yValidTimeDomain.shape)

(0,) (0,) (77628, 5655) (77628, 10)


In [25]:
fileStub = filepath.split('/')[-1].split('.')[0]

arraySavePath = '/blue/gkalamangalam/jmark.ettinger/predictScalp/preprocessed/timeDomain_%s_%s.npz' % (patient, fileStub)
np.savez(arraySavePath, 
         xTrainTimeDomain=xTrainTimeDomain, 
         xValidTimeDomain=xValidTimeDomain,
         yTrainTimeDomain=yTrainTimeDomain,
         yValidTimeDomain=yValidTimeDomain)
print(arraySavePath)

/blue/gkalamangalam/jmark.ettinger/predictScalp/preprocessed/timeDomain_UFSEEG031_Sz5File.npz


# STFT DATA

In [ ]:
# APPLY SHORT TERM FOURIER TRANSFORM TO THE DATA AND CHECK PARAMETERS FOR INVERTABILITY

secondsInSTFTWindow = .5
nperseg = subsampleFreq * secondsInSTFTWindow
noverlap = nperseg - 1
windowType = ('tukey', .25)

f, t, S = stft(dataSubsampled, fs=subsampleFreq, window=windowType, nperseg=nperseg, noverlap=noverlap, axis=0)

print('freq, ', 'time, ', 'stft shape')
print(f.shape, t.shape, S.shape)
print('inverse ok? ',check_NOLA(windowType, nperseg, noverlap))

In [ ]:
x_trainComplex = S[:, 0:-1, trainIndices].transpose([2,0,1])
y_trainComplex = S[:, -1, trainIndices].transpose()

x_validComplex = S[:, 0:-1, validationIndices].transpose([2,0,1])
y_validComplex = S[:, -1, validationIndices].transpose()

# MAKE REAL-VALUED TRAINING DATA BY CONVERTING STFT COMPLEX NUMBERS TO R,THETA
_,_,numCol = x_trainComplex.shape
x_trainRTheta = np.hstack([np.hstack([np.abs(x_trainComplex[:,:,i]), 
                                      np.angle(x_trainComplex[:,:,i])]) for i in range(numCol)])
x_validRTheta = np.hstack([np.hstack([np.abs(x_validComplex[:,:,i]), 
                                      np.angle(x_validComplex[:,:,i])]) for i in range(numCol)])

y_trainRTheta = np.hstack([np.abs(y_trainComplex), np.angle(y_trainComplex)])
y_validRTheta = np.hstack([np.abs(y_validComplex), np.angle(y_validComplex)])

_, nY = y_trainRTheta.shape
x_trainRTheta.shape, x_validRTheta.shape, y_trainRTheta.shape, y_validRTheta.shape

In [ ]:
# PLOT THE STFT OF A TIME SERIES (MAGNITUDE ONLY)

index = -1 # -1 is the target
vmax = .2

plt.figure()
plt.pcolormesh(t, f, np.abs(S[:,index,:]), shading='auto', cmap='hot', vmin=0, vmax=vmax)
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')
plt.title('Index: %s' % str(index))
plt.show()

In [ ]:
arraySavePath = '/blue/gkalamangalam/jmark.ettinger/predictScalp/freqRTheta_%s_%s_%s.npz' % (patient, targetScalpElectrode, mode)
np.savez(arraySavePath, 
         x_trainRTheta=x_trainRTheta, 
         x_validRTheta=x_validRTheta, 
         y_trainRTheta=y_trainRTheta, 
         y_validRTheta=y_validRTheta)

# COMBINE TIME AND FREQUENCY DOMAIN DATA

In [ ]:
x_trainTimeFreq = np.hstack([xTrainTimeDomain, x_trainRTheta])
y_trainTimeFreq = np.hstack([yTrainTimeDomain, y_trainRTheta])
x_validTimeFreq = np.hstack([xValidTimeDomain, x_validRTheta])
y_validTimeFreq = np.hstack([yValidTimeDomain, y_validRTheta])

In [ ]:
x_trainTimeFreq.shape, y_trainTimeFreq.shape, x_validTimeFreq.shape, y_validTimeFreq.shape

In [ ]:
arraySavePath = '/blue/gkalamangalam/jmark.ettinger/predictScalp/timeFreqRTheta_%s_%s_%s.npz' % (patient, targetScalpElectrode, mode)
np.savez(arraySavePath, 
         x_trainTimeFreq=x_trainTimeFreq, 
         x_validTimeFreq=x_validTimeFreq, 
         y_trainTimeFreq=y_trainTimeFreq, 
         y_validTimeFreq=y_validTimeFreq)

# SCRATCH

In [ ]:
# stft parameter tests

windowType = ('tukey', .25)

fakeData = np.random.rand(10000, 5)

f, t, S = stft(fakeData, fs=1000, window=windowType, nperseg=100, noverlap=0, axis=0, boundary=None)

f.shape, t.shape, S.shape, f, t

In [ ]:
# OLD VERSION FOR TIME DOMAIN

xTrainTimeDomain = dataSubsampled[trainIndices, 0:-1]
yTrainTimeDomain = np.expand_dims(dataSubsampled[trainIndices, -1], axis=1)

xValidTimeDomain = dataSubsampled[validationIndices, 0:-1]
yValidTimeDomain = np.expand_dims(dataSubsampled[validationIndices, -1], axis=1)

xTrainTimeDomain.shape, yTrainTimeDomain.shape, xValidTimeDomain.shape, yValidTimeDomain.shape